# Анализ бизнес-показателей приложения Procrastinate Pro+

**Описание проекта**

Вы — маркетинговый аналитик развлекательного приложения Procrastinate Pro+. Несмотря на огромные вложения в рекламу, последние несколько месяцев компания терпит убытки. Ваша задача — разобраться в причинах и помочь компании выйти в плюс.
Есть данные о пользователях, привлечённых с 1 мая по 27 октября 2019 года:
лог сервера с данными об их посещениях,
выгрузка их покупок за этот период,
рекламные расходы.

**Описание данных**

В распоряжении три датасета: 
- `visits_info_short.csv` хранит лог сервера с информацией о посещениях сайта, 
- `orders_info_short.csv` — информацию о заказах, 
- `costs_info_short.csv` — информацию о расходах на рекламу.

1. Структура visits_info_short.csv:
- `User Id` — уникальный идентификатор пользователя,
- `Region` — страна пользователя,
- `Device` — тип устройства пользователя,
- `Channel` — идентификатор источника перехода,
- `Session Start` — дата и время начала сессии,
- `Session End` — дата и время окончания сессии.

2. Структура orders_info_short.csv:
- `User Id` — уникальный идентификатор пользователя,
- `Event Dt` — дата и время покупки,
- `Revenue` — сумма заказа.

3. Структура costs_info_short.csv:
- `dt` — дата проведения рекламной кампании,
- `Channel` — идентификатор рекламного источника,
- `costs` — расходы на эту кампанию.

**Цели исследования:**

- Выяснить откуда приходят пользователи и какими устройствами они пользуются,
- Определить сколько стоит привлечение пользователей из различных рекламных каналов;
- Определить сколько денег приносит каждый клиент,
- Выяснить когда расходы на привлечение клиента окупаются,
- Определить какие факторы мешают привлечению клиентов.

**Ход работы:**

1. Обзор данных.
2. Предобработка данных.
3. Задать функции для расчёта и анализа LTV, ROI, удержания и конверсии.
4. Построение таблиц, отражающую количество пользователей и долю платящих из каждой страны, для каждого устройства и для каждого канала привлечения.
5. Проведение анализа расходов на маркетинг.
6. Проведение анализа окупаемости рекламы.
7. Написание общего вывода.


### Загрузите данные и подготовьте их к анализу

Загрузите данные о визитах, заказах и рекламных расходах из CSV-файлов в переменные.

**Пути к файлам**

- визиты: `/datasets/visits_info_short.csv`. [Скачать датасет](https://code.s3.yandex.net/datasets/visits_info_short.csv);
- заказы: `/datasets/orders_info_short.csv`. [Скачать датасет](https://code.s3.yandex.net/datasets/orders_info_short.csv);
- расходы: `/datasets/costs_info_short.csv`. [Скачать датасет](https://code.s3.yandex.net/datasets/costs_info_short.csv).

Изучите данные и выполните предобработку. Есть ли в данных пропуски и дубликаты? Убедитесь, что типы данных во всех колонках соответствуют сохранённым в них значениям. Обратите внимание на столбцы с датой и временем.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

In [2]:
# настройка отображения вещественных чисел в датасфрейме
pd.options.display.float_format = '{:.2f}'.format

In [3]:
try:
    visits, orders, costs = (
        pd.read_csv('datasets/visits_info_short.csv'), 
        pd.read_csv('datasets/orders_info_short.csv'), 
        pd.read_csv('datasets/costs_info_short.csv'), 
    )
except:
    visits, orders, costs = (
        pd.read_csv('https://code.s3.yandex.net/datasets/visits_info_short.csv'), 
        pd.read_csv('https://code.s3.yandex.net/datasets/orders_info_short.csv'), 
        pd.read_csv('https://code.s3.yandex.net/datasets/costs_info_short.csv'), 
    )

#### Знакомство с датасетами

In [4]:
visits.head()

,User Id,Region,Device,Channel,Session Start,Session End
0,981449118918,United States,iPhone,organic,2019-05-01 02:36:01,2019-05-01 02:45:01
1,278965908054,United States,iPhone,organic,2019-05-01 04:46:31,2019-05-01 04:47:35
2,590706206550,United States,Mac,organic,2019-05-01 14:09:25,2019-05-01 15:32:08
3,326433527971,United States,Android,TipTop,2019-05-01 00:29:59,2019-05-01 00:54:25
4,349773784594,United States,Mac,organic,2019-05-01 03:33:35,2019-05-01 03:57:40


In [5]:
visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309901 entries, 0 to 309900
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   User Id        309901 non-null  int64 
 1   Region         309901 non-null  object
 2   Device         309901 non-null  object
 3   Channel        309901 non-null  object
 4   Session Start  309901 non-null  object
 5   Session End    309901 non-null  object
dtypes: int64(1), object(5)
memory usage: 14.2+ MB


In [6]:
orders.head()

,User Id,Event Dt,Revenue
0,188246423999,2019-05-01 23:09:52,4.99
1,174361394180,2019-05-01 12:24:04,4.99
2,529610067795,2019-05-01 11:34:04,4.99
3,319939546352,2019-05-01 15:34:40,4.99
4,366000285810,2019-05-01 13:59:51,4.99


In [7]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40212 entries, 0 to 40211
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   User Id   40212 non-null  int64  
 1   Event Dt  40212 non-null  object 
 2   Revenue   40212 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 942.6+ KB


In [8]:
costs.head()

,dt,Channel,costs
0,2019-05-01,FaceBoom,113.30
1,2019-05-02,FaceBoom,78.10
2,2019-05-03,FaceBoom,85.80
3,2019-05-04,FaceBoom,136.40
4,2019-05-05,FaceBoom,122.10


In [9]:
costs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   dt       1800 non-null   object 
 1   Channel  1800 non-null   object 
 2   costs    1800 non-null   float64
dtypes: float64(1), object(2)
memory usage: 42.3+ KB


Во всех датасетах отсутствуют пропуски, данные полные.
Названия колонок не соответствуют "питоновскому" стандарту. Нужно перевести все названия в нижний регистр и заменить пробелы между словами на нижнее подчеркивание. 
Также необходимо столбцы с датой и временем привести к типу дата для дальнейшей работы.

#### Преобразование датасетов

In [10]:
# функция для переименования колонок
def renaming_columns(columns):
    columns = columns.str.lower().str.strip().str.replace(' ', '_')
    return columns

In [11]:
# преобразование названий колонок в "питоновский вид"
visits.columns = renaming_columns(visits.columns)
orders.columns = renaming_columns(orders.columns)
costs.columns = renaming_columns(costs.columns)

In [12]:
# преобразование данных о времени
visits['session_start'] = pd.to_datetime(visits['session_start'])
visits['session_end'] = pd.to_datetime(visits['session_end'])
orders['event_dt'] = pd.to_datetime(orders['event_dt'])
costs['dt'] = pd.to_datetime(costs['dt']).dt.date  

In [13]:
# вывод названий колонок
visits.columns

Index(['user_id', 'region', 'device', 'channel', 'session_start',
       'session_end'],
      dtype='object')

In [14]:
# знакомство с уникальными значениями некоторых столбцов
for col in ['region', 'device', 'channel']:
    print(col)
    print(visits[col].unique())
    print('-'*70)

region
['United States' 'UK' 'France' 'Germany']
----------------------------------------------------------------------
device
['iPhone' 'Mac' 'Android' 'PC']
----------------------------------------------------------------------
channel
['organic' 'TipTop' 'RocketSuperAds' 'YRabbit' 'FaceBoom' 'MediaTornado'
 'AdNonSense' 'LeapBob' 'WahooNetBanner' 'OppleCreativeMedia'
 'lambdaMediaAds']
----------------------------------------------------------------------


In [15]:
# проверка на дубликаты и пропуски
dataframes = {
    'visits': visits,
    'orders': orders,
    'costs': costs,
}

for name, df in dataframes.items():
    print(name)
    print('Количество дубликатов:', df.duplicated().sum())
    print('Количество пропусков:')
    print(df.isnull().sum ())
    print('-'*30)

visits
Количество дубликатов: 0
Количество пропусков:
user_id          0
region           0
device           0
channel          0
session_start    0
session_end      0
dtype: int64
------------------------------
orders
Количество дубликатов: 0
Количество пропусков:
user_id     0
event_dt    0
revenue     0
dtype: int64
------------------------------
costs
Количество дубликатов: 0
Количество пропусков:
dt         0
channel    0
costs      0
dtype: int64
------------------------------


In [16]:
# проверка случаев, когда окончание сессии было раньше чем её начало
visits.loc[visits['session_start'] > visits['session_end']]

,user_id,region,device,channel,session_start,session_end


In [17]:
# проверка суммы заказа на отрцательные числа, либо равных 0
orders.loc[orders['revenue'] <= 0]

,user_id,event_dt,revenue


In [18]:
# проверка расходов на маркетинг на отрцательные числа, либо равных 0
costs.loc[costs['costs'] <= 0]

,dt,channel,costs


Качество исходных данных хорошее. Нет дубликатов и пропусков. Сумма заказа и расходы на маркетинг больше нуля, что соответствует нашему требованию. Т.к. эти данные должны имееть цену.
Также все в порядке с началом и окончанием сессии: нет сессий, где окончание сессии было раньше чем её начало.

- Исправлены названия столбцов. 
- Колонки со времением преобразованы в тип данных datetime. 
- В уникальных значениях столбцов не обнаружены неявные дубликаты.

### Задайте функции для расчёта и анализа LTV, ROI, удержания и конверсии.

Разрешается использовать функции, с которыми вы познакомились в теоретических уроках.

Это функции для вычисления значений метрик:

- `get_profiles()` — для создания профилей пользователей,
- `get_retention()` — для подсчёта Retention Rate,
- `get_conversion()` — для подсчёта конверсии,
- `get_ltv()` — для подсчёта LTV.

А также функции для построения графиков:

- `filter_data()` — для сглаживания данных,
- `plot_retention()` — для построения графика Retention Rate,
- `plot_conversion()` — для построения графика конверсии,
- `plot_ltv_roi` — для визуализации LTV и ROI.

#### Функция для создания пользовательских профилей

In [19]:
def get_profiles(sessions, orders, ad_costs):

    # находим параметры первых посещений
    profiles = (
        sessions.sort_values(by=['user_id', 'session_start'])
        .groupby('user_id')
        .agg(
            {
                'session_start': 'first',
                'channel': 'first',
                'device': 'first',
                'region': 'first',
            }
        )
        .rename(columns={'session_start': 'first_ts'})
        .reset_index()
    )

    # для когортного анализа определяем дату первого посещения
    # и первый день месяца, в который это посещение произошло
    profiles['dt'] = profiles['first_ts'].dt.date
    profiles['month'] = profiles['first_ts'].astype('datetime64[M]')

    # добавляем признак платящих пользователей
    profiles['payer'] = profiles['user_id'].isin(orders['user_id'].unique())

    # считаем количество уникальных пользователей
    # с одинаковыми источником и датой привлечения
    new_users = (
        profiles.groupby(['dt', 'channel'])
        .agg({'user_id': 'nunique'})
        .rename(columns={'user_id': 'unique_users'})
        .reset_index()
    )

    # объединяем траты на рекламу и число привлечённых пользователей
    ad_costs = ad_costs.merge(new_users, on=['dt', 'channel'], how='left')

    # делим рекламные расходы на число привлечённых пользователей
    ad_costs['acquisition_cost'] = ad_costs['costs'] / ad_costs['unique_users']

    # добавляем стоимость привлечения в профили
    profiles = profiles.merge(
        ad_costs[['dt', 'channel', 'acquisition_cost']],
        on=['dt', 'channel'],
        how='left',
    )

    # стоимость привлечения органических пользователей равна нулю
    profiles['acquisition_cost'] = profiles['acquisition_cost'].fillna(0)

    return profiles


#### Функция для расчёта удержания

In [20]:
def get_retention(
    profiles,
    sessions,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # добавляем столбец payer в передаваемый dimensions список
    dimensions = ['payer'] + dimensions

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    # собираем «сырые» данные для расчёта удержания
    result_raw = result_raw.merge(
        sessions[['user_id', 'session_start']], on='user_id', how='left'
    )
    result_raw['lifetime'] = (
        result_raw['session_start'] - result_raw['first_ts']
    ).dt.days

    # функция для группировки таблицы по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        result = df.pivot_table(
            index=dims, columns='lifetime', values='user_id', aggfunc='nunique'
        )
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        result = result.div(result['cohort_size'], axis=0)
        result = result[['cohort_size'] + list(range(horizon_days))]
        result['cohort_size'] = cohort_sizes
        return result

    # получаем таблицу удержания
    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)

    # получаем таблицу динамики удержания
    result_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    # возвращаем обе таблицы и сырые данные
    return result_raw, result_grouped, result_in_time 

#### Функция для расчёта конверсии

In [21]:
def get_conversion(
    profiles,
    purchases,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    # определяем дату и время первой покупки для каждого пользователя
    first_purchases = (
        purchases.sort_values(by=['user_id', 'event_dt'])
        .groupby('user_id')
        .agg({'event_dt': 'first'})
        .reset_index()
    )

    # добавляем данные о покупках в профили
    result_raw = result_raw.merge(
        first_purchases[['user_id', 'event_dt']], on='user_id', how='left'
    )

    # рассчитываем лайфтайм для каждой покупки
    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days

    # группируем по cohort, если в dimensions ничего нет
    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users' 
        dimensions = dimensions + ['cohort']

    # функция для группировки таблицы по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        result = df.pivot_table(
            index=dims, columns='lifetime', values='user_id', aggfunc='nunique'
        )
        result = result.fillna(0).cumsum(axis = 1)
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        # делим каждую «ячейку» в строке на размер когорты
        # и получаем conversion rate
        result = result.div(result['cohort_size'], axis=0)
        result = result[['cohort_size'] + list(range(horizon_days))]
        result['cohort_size'] = cohort_sizes
        return result

    # получаем таблицу конверсии
    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)

    # для таблицы динамики конверсии убираем 'cohort' из dimensions
    if 'cohort' in dimensions: 
        dimensions = []

    # получаем таблицу динамики конверсии
    result_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    # возвращаем обе таблицы и сырые данные
    return result_raw, result_grouped, result_in_time 

#### Функция для расчёта LTV и ROI

In [22]:
def get_ltv(
    profiles,
    purchases,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')
    # добавляем данные о покупках в профили
    result_raw = result_raw.merge(
        purchases[['user_id', 'event_dt', 'revenue']], on='user_id', how='left'
    )
    # рассчитываем лайфтайм пользователя для каждой покупки
    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days
    # группируем по cohort, если в dimensions ничего нет
    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users'
        dimensions = dimensions + ['cohort']

    # функция группировки по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        # строим «треугольную» таблицу выручки
        result = df.pivot_table(
            index=dims, columns='lifetime', values='revenue', aggfunc='sum'
        )
        # находим сумму выручки с накоплением
        result = result.fillna(0).cumsum(axis=1)
        # вычисляем размеры когорт
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        # объединяем размеры когорт и таблицу выручки
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        # считаем LTV: делим каждую «ячейку» в строке на размер когорты
        result = result.div(result['cohort_size'], axis=0)
        # исключаем все лайфтаймы, превышающие горизонт анализа
        result = result[['cohort_size'] + list(range(horizon_days))]
        # восстанавливаем размеры когорт
        result['cohort_size'] = cohort_sizes

        # собираем датафрейм с данными пользователей и значениями CAC, 
        # добавляя параметры из dimensions
        cac = df[['user_id', 'acquisition_cost'] + dims].drop_duplicates()

        # считаем средний CAC по параметрам из dimensions
        cac = (
            cac.groupby(dims)
            .agg({'acquisition_cost': 'mean'})
            .rename(columns={'acquisition_cost': 'cac'})
        )

        # считаем ROI: делим LTV на CAC
        roi = result.div(cac['cac'], axis=0)

        # удаляем строки с бесконечным ROI
        roi = roi[~roi['cohort_size'].isin([np.inf])]

        # восстанавливаем размеры когорт в таблице ROI
        roi['cohort_size'] = cohort_sizes

        # добавляем CAC в таблицу ROI
        roi['cac'] = cac['cac']

        # в финальной таблице оставляем размеры когорт, CAC
        # и ROI в лайфтаймы, не превышающие горизонт анализа
        roi = roi[['cohort_size', 'cac'] + list(range(horizon_days))]

        # возвращаем таблицы LTV и ROI
        return result, roi

    # получаем таблицы LTV и ROI
    result_grouped, roi_grouped = group_by_dimensions(
        result_raw, dimensions, horizon_days
    )

    # для таблиц динамики убираем 'cohort' из dimensions
    if 'cohort' in dimensions:
        dimensions = []

    # получаем таблицы динамики LTV и ROI
    result_in_time, roi_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    return (
        result_raw,  # сырые данные
        result_grouped,  # таблица LTV
        result_in_time,  # таблица динамики LTV
        roi_grouped,  # таблица ROI
        roi_in_time,  # таблица динамики ROI
    ) 

#### Функция для сглаживания фрейма

In [23]:
def filter_data(df, window):
    # для каждого столбца применяем скользящее среднее
    for column in df.columns.values:
        df[column] = df[column].rolling(window).mean() 
    return df 

#### Функция для визуализации удержания

In [24]:
def plot_retention(retention, retention_history, horizon, window=7):

    # задаём размер сетки для графиков
    plt.figure(figsize=(15, 10))

    # исключаем размеры когорт и удержание первого дня
    retention = retention.drop(columns=['cohort_size', 0])
    # в таблице динамики оставляем только нужный лайфтайм
    retention_history = retention_history.drop(columns=['cohort_size'])[
        [horizon - 1]
    ]

    # если в индексах таблицы удержания только payer,
    # добавляем второй признак — cohort
    if retention.index.nlevels == 1:
        retention['cohort'] = 'All users'
        retention = retention.reset_index().set_index(['cohort', 'payer'])

    # в таблице графиков — два столбца и две строки, четыре ячейки
    # в первой строим кривые удержания платящих пользователей
    ax1 = plt.subplot(2, 2, 1)
    retention.query('payer == True').droplevel('payer').T.plot(
        grid=True, ax=ax1
    )
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('Удержание платящих пользователей')

    # во второй ячейке строим кривые удержания неплатящих
    # вертикальная ось — от графика из первой ячейки
    ax2 = plt.subplot(2, 2, 2, sharey=ax1)
    retention.query('payer == False').droplevel('payer').T.plot(
        grid=True, ax=ax2
    )
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('Удержание неплатящих пользователей')

    # в третьей ячейке — динамика удержания платящих
    ax3 = plt.subplot(2, 2, 3)
    # получаем названия столбцов для сводной таблицы
    columns = [
        name
        for name in retention_history.index.names
        if name not in ['dt', 'payer']
    ]
    # фильтруем данные и строим график
    filtered_data = retention_history.query('payer == True').pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax3)
    plt.xlabel('Дата привлечения')
    plt.title(
        'Динамика удержания платящих пользователей на {}-й день'.format(
            horizon
        )
    )

    # в чётвертой ячейке — динамика удержания неплатящих
    ax4 = plt.subplot(2, 2, 4, sharey=ax3)
    # фильтруем данные и строим график
    filtered_data = retention_history.query('payer == False').pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax4)
    plt.xlabel('Дата привлечения')
    plt.title(
        'Динамика удержания неплатящих пользователей на {}-й день'.format(
            horizon
        )
    )
    
    plt.tight_layout()
    plt.show() 

#### Функция для визуализации конверсии

In [25]:
def plot_conversion(conversion, conversion_history, horizon, window=7):

    # задаём размер сетки для графиков
    plt.figure(figsize=(15, 5))

    # исключаем размеры когорт
    conversion = conversion.drop(columns=['cohort_size'])
    # в таблице динамики оставляем только нужный лайфтайм
    conversion_history = conversion_history.drop(columns=['cohort_size'])[
        [horizon - 1]
    ]

    # первый график — кривые конверсии
    ax1 = plt.subplot(1, 2, 1)
    conversion.T.plot(grid=True, ax=ax1)
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('Конверсия пользователей')

    # второй график — динамика конверсии
    ax2 = plt.subplot(1, 2, 2, sharey=ax1)
    columns = [
        # столбцами сводной таблицы станут все столбцы индекса, кроме даты
        name for name in conversion_history.index.names if name not in ['dt']
    ]
    filtered_data = conversion_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax2)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика конверсии пользователей на {}-й день'.format(horizon))

    plt.tight_layout()
    plt.show() 

#### Функция для визуализации LTV и ROI

In [26]:
# функция для визуализации LTV и ROI

def plot_ltv_roi(ltv, ltv_history, roi, roi_history, horizon, window=7):

    # задаём сетку отрисовки графиков
    plt.figure(figsize=(20, 10))

    # из таблицы ltv исключаем размеры когорт
    ltv = ltv.drop(columns=['cohort_size'])
    # в таблице динамики ltv оставляем только нужный лайфтайм
    ltv_history = ltv_history.drop(columns=['cohort_size'])[[horizon - 1]]

    # стоимость привлечения запишем в отдельный фрейм
    cac_history = roi_history[['cac']]

    # из таблицы roi исключаем размеры когорт и cac
    roi = roi.drop(columns=['cohort_size', 'cac'])
    # в таблице динамики roi оставляем только нужный лайфтайм
    roi_history = roi_history.drop(columns=['cohort_size', 'cac'])[
        [horizon - 1]
    ]

    # первый график — кривые ltv
    ax1 = plt.subplot(2, 3, 1)
    ltv.T.plot(grid=True, ax=ax1)
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('LTV')

    # второй график — динамика ltv
    ax2 = plt.subplot(2, 3, 2, sharey=ax1)
    # столбцами сводной таблицы станут все столбцы индекса, кроме даты
    columns = [name for name in ltv_history.index.names if name not in ['dt']]
    filtered_data = ltv_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax2)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика LTV пользователей на {}-й день'.format(horizon))

    # третий график — динамика cac
    ax3 = plt.subplot(2, 3, 3, sharey=ax1)
    # столбцами сводной таблицы станут все столбцы индекса, кроме даты
    columns = [name for name in cac_history.index.names if name not in ['dt']]
    filtered_data = cac_history.pivot_table(
        index='dt', columns=columns, values='cac', aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax3)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика стоимости привлечения пользователей')

    # четвёртый график — кривые roi
    ax4 = plt.subplot(2, 3, 4)
    roi.T.plot(grid=True, ax=ax4)
    plt.axhline(y=1, color='red', linestyle='--', label='Уровень окупаемости')
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('ROI')

    # пятый график — динамика roi
    ax5 = plt.subplot(2, 3, 5, sharey=ax4)
    # столбцами сводной таблицы станут все столбцы индекса, кроме даты
    columns = [name for name in roi_history.index.names if name not in ['dt']]
    filtered_data = roi_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax5)
    plt.axhline(y=1, color='red', linestyle='--', label='Уровень окупаемости')
    plt.xlabel('Дата привлечения')
    plt.title('Динамика ROI пользователей на {}-й день'.format(horizon))

    plt.tight_layout()
    plt.show() 

<div class="alert alert-block alert-success">✔️
    

__Комментарий от ревьюера №1__
    
Хорошо, все необходимые функции были заданы, можно приступать к расчета и анализу👀

### Исследовательский анализ данных

- Составьте профили пользователей. Определите минимальную и максимальную даты привлечения пользователей.
- Выясните, из каких стран пользователи приходят в приложение и на какую страну приходится больше всего платящих пользователей. Постройте таблицу, отражающую количество пользователей и долю платящих из каждой страны.
- Узнайте, какими устройствами пользуются клиенты и какие устройства предпочитают платящие пользователи. Постройте таблицу, отражающую количество пользователей и долю платящих для каждого устройства.
- Изучите рекламные источники привлечения и определите каналы, из которых пришло больше всего платящих пользователей. Постройте таблицу, отражающую количество пользователей и долю платящих для каждого канала привлечения.

После каждого пункта сформулируйте выводы.

#### Профили пользователей

In [27]:
# создание профилей пользователей
profiles = get_profiles(visits, orders, costs)
profiles.head()

TypeError: Cannot cast DatetimeArray to dtype datetime64[M]

In [ ]:
# вывод максимальной даты привлечения пользователей
profiles['first_ts'].dt.date.max()

In [ ]:
# вывод минимальной даты привлечения пользователей
profiles['first_ts'].dt.date.min()

Есть данные о пользователях, привлеченных с 1 мая по 27 октября 2019 года.

#### Доля платящих пользователей из каждой страны

In [ ]:
# функция для нахождения доли платящих пользователей по нужной колонке
def paying_users(columns):
    profiles_ = profiles.groupby(columns).agg({'user_id':'nunique', 'payer':['sum', 'mean']})                
    profiles_.columns = ['id_unique', 'count_payer', 'percent_payer']
    profiles_['percent_payer'] = profiles_['percent_payer'] * 100
    
    return profiles_.sort_values('percent_payer', ascending=False)
    

In [ ]:
profiles_payer_region = paying_users('region')
profiles_payer_region

- Несомненным лидиром по продажам и долей платящих пользователей является United States. 
- Примерно на 3% меньше доля у остальных стран Germany, UK, France, что составляет разницу чуть ли не в 2 раза меньше чем у United States. А количество платящих вообще в 10 раз меньше.
- Между  странами Европы доля платящих пользователей и их количество распределенно примерно равномерно, отличие в долях составляет 0,1-0,3%.

#### Доля платящих пользователей для каждого устройства

In [ ]:
profiles_payer_device = paying_users('device')
profiles_payer_device

- Самая большая доля платящих пользователей приходится на Mac. 
- iPhone занимает второе место уступая всего 0,15%. Зато по количеству платящих пользователей iPhone лидирует - в 1,5-2 выше, чем у других устройств.
- Android находится на третьем месте, уступая Mac 0,5%. Но по количеству привлеченных пользователей примерно одинаково с Mac и РС.
- Последнее место занимает PC и по доле привлеченных пользователей и по количеству платящих. У PC разница с лидером в доле составляет 1,3%.


#### Доля платящих пользователей для каждого канала привлечения

In [ ]:
profiles_payer_channel = paying_users('channel')
profiles_payer_channel

In [ ]:
sns.set() 
profiles_payer_channel['percent_payer'].plot(
       kind='barh', 
       grid=True,  
       legend=False, 
       figsize=(10, 5),
       xlabel='Канал привлечения',
       title='Доля платящих пользователей для каждого канала привлечения');


- Лидирами по долям платящих являются FaceBoom, AdNonSense, lambdaMediaAds. У этих каналах более 10% доля платящих. Но FaceBoom лидирует также и по количеству платящих - более 3500 против 440 (AdNonSense) и 225 (lambdaMediaAds).

- "Среднее звено" занимают каналы TipTop, RocketSuperAds, WahooNetBanner. У них примерно от 5 до 10% доли. Здесь TipTop "возглавляет" тройку каналов по доле и по количеству платящих - более 1800 против 350 (RocketSuperAds) и 450 (WahooNetBanner).

- Менее 4% доли платящих у каналов YRabbit, MediaTornado, LeapBob, OppleCreativeMedia, organic. Но есть интересных факт - хоть у organic меньше всего доля платящих, зато больше всего количество платящих пользователей из этих каналов. У organic 1160 платящих пользователей, а у остальных количество составляет всего примерно 150-260.  

### Маркетинг

- Посчитайте общую сумму расходов на маркетинг.
- Выясните, как траты распределены по рекламным источникам, то есть сколько денег потратили на каждый источник.
- Постройте визуализацию динамики изменения расходов во времени (по неделям и месяцам) по каждому источнику. Постарайтесь отразить это на одном графике.
- Узнайте, сколько в среднем стоило привлечение одного пользователя (CAC) из каждого источника. Используйте профили пользователей.

Напишите промежуточные выводы.

#### Общая сумма расходов на маркетинг

In [ ]:
total_cost = profiles['acquisition_cost'].sum()
round(total_cost, 2)

#### Распределение трат по рекламным источникам

In [ ]:
# функция для создания сводной таблицы сгруппированной по каналам привлечения 
def channel_costs(aggfunc):
    profiles_channel = (profiles
                    .pivot_table(index='channel', values='acquisition_cost', aggfunc=aggfunc)
                    .sort_values('acquisition_cost', ascending=False))
    return profiles_channel

In [ ]:
# удаление organic, т.к. этот канал не входит в бюджет
profiles = profiles.query('channel != "organic"')

In [ ]:
# распределение трат по рекламным источникам
channel_total_cost = channel_costs('sum')
channel_total_cost['percent_cost'] = channel_total_cost['acquisition_cost'] / total_cost * 100
channel_total_cost 

- Больше половины расходов на маркетинг "ушло" на TipTop. 
- FaceBoom "забрал" почти треть расходов.
- У остальных 8 каналов расходы более чем в 10 раз меньше TipTop. 

- На маркетинг каналов WahooNetBanner, AdNonSense и OppleCreativeMedia в общем израсходовано примерно 11000.
- Расходы на маркетинг RocketSuperAds, LeapBob и lambdaMediaAds составили менее 2000 на каждый канал.
- И менее 1000 обошлись каналы MediaTornado и YRabbit.

#### Визуализация динамики изменения расходов во времени 

In [ ]:
# создание новых столбцов для дальнейшей работы 
profiles['week'] = profiles['first_ts'].dt.isocalendar().week
profiles['month'] = profiles['first_ts'].dt.month

In [ ]:
profiles.head()

In [ ]:

fig, ax = plt.subplots(1, 2, figsize=(15,8))
fig.suptitle('Динамики изменения расходов во времени по каждому источнику')

for i, col in enumerate(['month', 'week']):
    (profiles
     .pivot_table(index=col, columns='channel', values='acquisition_cost', aggfunc='sum')
     .plot(ylim=(0,15000), ax=ax[i], ylabel='Расходы в у.е.'))

- На TipTop самые высокие расходы, которые с каждым месяцем увеличиваются начиная с отметки 3500 в мае и достигают примерно 13000 в сентябре. 
- Тоже высокие траты у FaceBoom - примерно с 3800 в мае до 5800 в июне, далее немного повышается к августу. Потом остается примерно на одном уровне около 6000.
- У остальных источников привлечения динамика расходов распределена равномерно по времени и составляет примерно менее 1000 в месяц на протяжении всего периода.

#### Средняя стоимость привлечение одного пользователя (CAC) 

In [ ]:
# cредняя стоимость привлечение одного пользователя (CAC)
round(total_cost / profiles['user_id'].nunique(), 2)

In [ ]:
# cредняя стоимость привлечение одного пользователя (CAC) из каждого источника
channel_costs('mean')

- Самый затратный источник - TipTop. Он превышает больше чем в 2 раза cреднюю стоимость привлечение одного пользователя. И выше более в 2 раза чем остальные источники привлечения пользователей.
- У FaceBoom и AdNonSense затраты на одного пользователя составили чуть выше 1. 
- У RocketSuperAds, WahooNetBanner и lambdaMediaAds стоимость привлечения в диапазоне от 0,41 до 0,72
- Самые малозатратные источники - OppleCreativeMedia, YRabbit, MediaTornado, LeapBob. У этих каналов  стоимость привлечения одного пользователя составила менее 0,26.

### Оцените окупаемость рекламы

Используя графики LTV, ROI и CAC, проанализируйте окупаемость рекламы. Считайте, что на календаре 1 ноября 2019 года, а в бизнес-плане заложено, что пользователи должны окупаться не позднее чем через две недели после привлечения. Необходимость включения в анализ органических пользователей определите самостоятельно.

- Проанализируйте окупаемость рекламы c помощью графиков LTV и ROI, а также графики динамики LTV, CAC и ROI.
- Проверьте конверсию пользователей и динамику её изменения. То же самое сделайте с удержанием пользователей. Постройте и изучите графики конверсии и удержания.
- Проанализируйте окупаемость рекламы с разбивкой по устройствам. Постройте графики LTV и ROI, а также графики динамики LTV, CAC и ROI.
- Проанализируйте окупаемость рекламы с разбивкой по странам. Постройте графики LTV и ROI, а также графики динамики LTV, CAC и ROI.
- Проанализируйте окупаемость рекламы с разбивкой по рекламным каналам. Постройте графики LTV и ROI, а также графики динамики LTV, CAC и ROI.
- Ответьте на такие вопросы:
    - Окупается ли реклама, направленная на привлечение пользователей в целом?
    - Какие устройства, страны и рекламные каналы могут оказывать негативное влияние на окупаемость рекламы?
    - Чем могут быть вызваны проблемы окупаемости?

Напишите вывод, опишите возможные причины обнаруженных проблем и промежуточные рекомендации для рекламного отдела.

#### Окупаемость рекламы c помощью графиков LTV и ROI

In [ ]:
observation_date = datetime(2019, 11, 1).date()  # момент анализа
horizon_days = 14  # горизонт анализа 

In [ ]:
# подсчет LTV и ROI
ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles, orders, observation_date, horizon_days
) 

In [ ]:
# построение графиков LTV, ROI, CAC
plot_ltv_roi(ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days)

- Реклама не окупается. ROI в конце двух недель около 80%. В мае окупаемость инвестиций имеет самые высокие показатели - выше 100%. С середины июня ROI находится ниже порога окупаемости.
- CAC вырос почти в 2 раза с первоначальной стоимости. Возможно дело в увеличении рекламного бюджета.
- На LTV влияет сезонный фактор, но этот показатель достаточно стабилен. Значит, дело не в ухудшении качества пользователей.

#### Конверсия и удержание пользователей

In [ ]:
def visualization_conversion_retention(dimensions=[]):
    # рассчет конверсии
    conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles, orders, observation_date, horizon_days, dimensions=dimensions)
    # рассчет удержания
    retention_raw, retention_grouped, retention_history = get_retention(
    profiles, visits, observation_date, horizon_days, dimensions=dimensions)
    
    # визуализация конверсии и удержания   
    plot_conversion(conversion_grouped, conversion_history, horizon_days) 
    plot_retention(retention_grouped, retention_history, horizon_days) 


In [ ]:
print('Конверсия и удержание пользователей')
visualization_conversion_retention()

Судя по графикам, пользователи конвертируются хорошо и постоянно. Удержание также стабильно.

#### Oкупаемость рекламы с разбивкой по устройствам

In [ ]:
dimensions = ['device']

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles, orders, observation_date, horizon_days, dimensions=dimensions
)

In [ ]:
# визуализация окупаемости с разбивкой по устройствам
plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=14
) 

- LTV стабильна и почти одинакова для всех устройств.
- Но реклама окупается только для РС, хотя LTV ниже, чем у других устройств. Это связано с низкой стоимостью привлечения.
- У iPhone и Mac стоимость привлечения выше, чем у остальных, а ROI самая низкая.

#### Oкупаемость рекламы с разбивкой по странам

In [ ]:
# визуализация окупаемости с разбивкой по странам
dimensions = ['region']

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=14
) 

- LTV всё так же подвержен сезонности, но стабилен. 
- В Европе реклама окупается уже на 4 - 6 неделе. Стоимость привлечения в июне снижается на 1/3 и остается на одном уровне за весь период.
- LTV в United States выше, чем в Европе. Но из-за высокой стоимости привлечения клиентов (более чем в 4 раза по сравнению с Европой) реклама в United States не окупается.

#### Oкупаемость рекламы с разбивкой по рекламным каналам

In [ ]:
# визуализация окупаемости с разбивкой по рекламным каналам
dimensions = ['channel']

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=14
) 

- У TipTop, FaceBoom и AdNonSense реклама не окупается. У этих же каналов самая высокая стоимость привлечения. 
- У остальных каналов реклама окупается и находится примерно на одном уровне. Чуть ниже ROI только у OppleCreativeMedia и WahooNetBanner.
- У TipTop САС растет с каждым месяцем и в конце периода выше более чем в 3 раза остальных каналов.
- У остальных источников стоимость привлечения стабильна за весь период. 
- Лучшие показатели LTV у lambdaMediaAds и TipTop. У канала lambdaMediaAds средняя САС.
- Самые низкие показатели LTV у каналов OppleCreativeMedia, LeapBob, MediaTornado, YRabbit. У этих же каналов самая низкая стоимость привлечения.	

#### Конверсия и удержание пользователей с разбивкой по устройствам

In [ ]:
print('Конверсия и удержание пользователей с разбивкой по устройствам')
visualization_conversion_retention(['device'])

- Хуже всего конвертируются пользователи РС. Mac и iPhone лидируют по конверсии.Немного от них отстает Android. 
- На графиках удержания всё наоборот: пользователи РС удерживаются лучше, а у Mac и iPhone самые низкие показатели. 
- В целом показатели не сильно отличаются друг от друга. Аномальных значений нет.

#### Конверсия и удержание пользователей с разбивкой по странам

In [ ]:
print('Конверсия и удержание пользователей с разбивкой по странам')
visualization_conversion_retention(['region'])

Конверсия США примерно в 2 раза больше стран Европы. Но удержание заметно ниже остальных стран.

#### Конверсия и удержание пользователей с разбивкой по каналам привлечения

In [ ]:
print('Конверсия и удержание пользователей с разбивкой по каналам привлечения')
visualization_conversion_retention(['channel'])

- Самая лучшая конвертация у каналов FaceBoom, AdNonSense. Но у них же самое низкое удержание.
- У остальных каналов удержание находится примерно на одном уровне
- Самый низкий показатель конверсии у OppleCreativeMedia, LeapBob, MediaTornado и YRabbit.

### Напишите выводы

- Выделите причины неэффективности привлечения пользователей.
- Сформулируйте рекомендации для отдела маркетинга.

#### Общие выводы

**Общие метрики**
- Пользователи конвертируются хорошо и постоянно. Удержание тоже стабильно. 
- На LTV влияет сезонный фактор, но этот показатель достаточно стабилен. Значит, дело не в ухудшении качества пользователей.
- ROI в конце двух недель около 80%. В мае окупаемость инвестиций имеет самые высокие показатели - выше 100%. С середины июня ROI падает ниже порога окупаемости.
- CAC вырос почти в 2 раза с первоначальной стоимости.

**Метрики по странам**
- Несомненным лидиром по доле платящих пользователей является США, что составляет разницу почти в 2 раза больше чем у стран Европы. А количество платящих более чем в 3 раз больше всей Европы.
- В Европе реклама окупается уже на 4 - 6 неделе. 
- LTV США выше чем у стран Европы, но реклама не окупается из-за высокой стоимости привлечения, которая в 4 раза выше чем у стран Европы. ROI "дотягивает" только до 70%.
- Конверсия США примерно в 2 раза больше стран Европы. Но удержание заметно ниже остальных стран.

**Метрики по устройствам**
- Доля платящих пользователей примерно у всех одинаковая.
- LTV стабильна и почти одинакова для всех устройств.
- Но реклама окупается только для РС, хотя LTV ниже, чем у других устройств. Это связано с низкой стоимостью привлечения.
- У iPhone и Mac стоимость привлечения выше, чем у остальных, а ROI самая низкая.
- В целом показатели конверсии и удержания не сильно отличаются друг от друга.

**Метрики по каналам привлечения**

`FaceBoom:`
- Лидер по доле платящих клиентов (12.2).
- На втором месте по cредней стоимости привлечение одного пользователя (1.11), но почти в 3 раза меньше чем у TipTop.
- Израсховано треть бюджета на маркетинг. Начиная с 3800 в мае повышается к августу. Далее остается примерно на одном уровне около 6000.
- По LTV занимает среднюю позицию среди каналов.
- Самая высокая конверсия, но самый низкий показатель удержания.
- Реклама не окупается.

`AdNonSense:`
- Второе место по доле платящих клиентов (11.34).
- Третье место по cредней стоимости привлечение одного пользователя (1.01).
- На маркетинг израсходовано менее 4% бюджета. Распределение равномерное на весь период.
- По LTV занимает среднюю позицию среди каналов.
- Высокая конверсия (2 место), но очень низкий показатель удержания.
- Реклама не окупается.

`lambdaMediaAds:`
- Третье место по доле платящих клиентов (10.47).
- Четвертое место по cредней стоимости привлечение одного пользователя (0.72).
- На маркетинг израсходовано менее 1.5% бюджета. Распределение равномерное на весь период.
- Лидер по LTV.
- Высокая конверсия (3 место), высокий показатель удержания.
- Реклама окупается, один из высоких показателей ROI.

`TipTop:`
- По доле платящих клиентов занимает четвертое место (9.6).
- Самая высокая cредняя стоимость привлечение одного пользователя (2.8), более в 2 раза больше чем у остальных источников. 
- Самые высокие расходы, которые с каждым месяцем увеличиваются начиная с отметки 3500 в мае и достигают примерно 13000 в сентябре. На маркетинг израсходовано больше 50% от общего бюджета.
- По LTV занимает второе место, уступая lambdaMediaAds.
- Высокая конверсия (4 место), хороший показатель удержания.
- Реклама не окупается, самый низкий показатель ROI.

`Остальные источники:`
- Доля платящих клиентов менее 8%.
- Средняя стоимость привлечение одного пользователя составляет от 0.2 до 0.6
- Стоимость привлечения стабильна за весь период и составляет чуть более 12% общего бюджета на все каналы. 
- Самые низкие показатели LTV у каналов OppleCreativeMedia, LeapBob, MediaTornado, YRabbit. 	
- Реклама окупается и находится примерно на одном уровне. Чуть ниже ROI только у OppleCreativeMedia и WahooNetBanner.

#### Рекомендации для отдела маркетинга

1. Хотя у США высокий показатель LTV и доля привлечения пользователей, но из-за высокой стоимости привлечения клиентов и низкого показателя удержания пользователей реклама не окупается. Рекомендовано снизить САС на United States. 


2. Переоценён канал TipTop. Реклама не окупается, самый низкий показатель ROI, т.к. у этого источника самая высокая cредняя стоимость привлечение одного пользователя и самые высокие расходы на маркетинг. Рекомендовано перенаправить деньги на более выгодный канал.


3. Недооценён канал lambdaMediaAds, у которого высокая доля платящих клиентов и невысокая стоимость привлечение одного пользователя. У этого канала один из высоких показателей ROI, но очень маленькая доля бюджета на маркетинг. Рекомендовано повысить бюджет на рекламу.